# Land Use Categorization for TMD23
This is intended to be a demo implementing the new definitions of land use categorizations for the Trip Generation step of the TDM23 (travel demand model in development). This notebook starts with categorization of Central Business Districts (CBDs).


Started March 30th, 2021 - Margaret Atkinson

#### Requirements:
This is running as a ipynb in Jupyter Notebooks in an Anaconda environment that contains the following:
- Python 3.7
- numpy
- pandas
- openmatrix
- matplotlib
- descartes
- ipympl
- geopandas
- nb_conda
- nb_conda_kernels
- folium

For more information on how this environment was set up, see: https://github.com/bkrepp-ctps/mde-prototype-python


#### Links about Decay Functions
- https://aceso.readthedocs.io/_/downloads/en/latest/pdf/
- https://geographicdata.science/book/notebooks/04_spatial_weights.html
- https://github.com/pysal/pysal


## Definitions:
### CBD
CBD: A TAZ is marked as a CBD if it is within 0.25 miles of two heavy rail (subway) stops from two different lines.

Methodology Plan: Buffer each heavy rail stop and dissolve by line to get one buffer per heavy rail line. Count the number of heavy rail line buffers intersect with each TAZ. If number is >1, the TAZ is a CBD.

### Urban

### Suburban

### Rural

In [1]:
#IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [16]:
#FOR CBD - some of this can be reused
#Import Data
#import TAZ shapefile
#(https://geopandas.org/docs/user_guide/io.html)
taz = "G:\Data_Resources\DataStore\Ben_Marg_Proto\candidate_CTPS_TAZ_STATE_2019.shp"
    #load into a geodataframe
taz = gpd.read_file(taz)
    #filter to just be TAZ ID and geometry
taz = taz[['id','geometry']]
    #rename ID to TAZ_ID
taz = taz.rename(columns={"id":"TAZ_ID"})

#import all stops (must have line column)


#transform TAZ and stop layers' CRS to CTPS Standard - "EPSG:26986" (Massachusetts State Plane)
taz = taz.to_crs({"init": "EPSG:26986"})
#hr_stops = hr_stops.to_crs({"init": "EPSG:26986"})

#filter all stops to just heavy rail stops
#hr_stops = hr_stops[hr_stops['Line_ID'] in ['Red', 'Blue', 'Orange']]

taz.head()

,TAZ_ID,geometry
0,4398,"POLYGON ((246729.399 855385.952, 246722.789 85..."
1,2571,"POLYGON ((247846.936 856739.950, 247846.689 85..."
2,2669,"POLYGON ((249688.220 857104.527, 249688.219 85..."
3,4392,"POLYGON ((251073.143 851003.537, 251079.003 85..."
4,2641,"POLYGON ((264256.501 872054.207, 264263.627 87..."


## CBD

In [ ]:
#BUFFER CELL

#convert the 0.25 mile buffer distance into meters (CRS is in meters)
buf25 = 0.25/1609.34

#Buffer the heavy rail stops
#(https://geopandas.org/docs/user_guide/geometric_manipulations.html)
hr_buf = hr_stops.buffer(buf25)

#group by line and dissolve
#(https://geopandas.org/docs/user_guide/aggregation_with_dissolve.html)
hr_buf_dis = hr_buf.dissolve(by='Line_ID')

In [ ]:
#INTERSECT AND COUNT CELL

#do a spatial join between TAZ and buffers
#(https://geopandas.org/docs/user_guide/mergingdata.html)
hr_sj = gpd.sjoin(taz,hr_buf_dis,how ="inner", op="intersect")
#make pivot table counting for each TAZ ID how many Line_IDs
    #note each line should only have one polygon so a taz should not be able to have multiple intersections with the same LineID
    #but check this
#(https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html)
hr_pv = pd.pivot_table(hr_sj, values = "TAZ_ID", aggfunc={'Line_ID':np.count}, fill_value = 0)
hr_pv

In [ ]:
#join pivot back to geo?

#rename Line_ID count field
hr_join = hr_join.rename(columns={"Line_ID":"Line_Count"})

#make CBD flag field (conditional - 1 where CBD, 0 where not)
#(https://cmdlinetips.com/2019/05/how-to-create-a-column-using-condition-on-another-column-in-pandas/)
hr_join['CBD_Flag'] = np.where(hr_join.Line_Count >1, 1, 0)

#filter to just have CBD TAZs
#(https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/)
CBD_TAZ = hr_join[hr_join['CBD'] = 1]

hr_join.head()

In [ ]:
#show a map of the result symbolized on CBD flag field.
hr_join.plot("CBD", figsize=(10.0,8.0), legend=True)


In [ ]:
#output as shapefile
# Output path
outfp = r"C:\Users\matkinson\Downloads\TAZ_CBD.shp"

# Save to disk
CBD_TAZ.to_file(outfp)